In [5]:
!pip install onnx onnxruntime

In [6]:
import math
import onnx
import onnxruntime as ort
import numpy as np
from onnx import TensorProto

# Define input and output tensor names
input1_name = "X"
maxpool_output_name1 = "Y"
maxpool_output_name2 = "Indices"


# Create the ONNX model with maxpool operator
def create_maxpool_model(input_shape, output_shape, dtype, paddings, k_shape):

    #Create "input-rank" input tensor
    input1 = onnx.helper.make_tensor_value_info(input1_name, dtype, input_shape)

    # Create output tensor (final result after maxpool operation)
    maxpool_output1 = onnx.helper.make_tensor_value_info(maxpool_output_name1, dtype, output_shape)
    maxpool_output2 = onnx.helper.make_tensor_value_info(maxpool_output_name2, TensorProto.INT64, output_shape)

    # Define clip node
    maxpool_node = onnx.helper.make_node(
        "MaxPool",
        inputs=[input1_name],
        outputs=[maxpool_output_name1, maxpool_output_name2],
        ceil_mode = 0,
        dilations=[1,1],
        kernel_shape=k_shape,
        pads=paddings,
        strides=[1, 1],
        auto_pad='NOTSET',
    )

    # Create the ONNX graph
    graph_def = onnx.helper.make_graph(
        [maxpool_node],
        "Test_MaxPool",
        [input1],
        [maxpool_output1, maxpool_output2],
    )

    # Create the ONNX model
    model = onnx.helper.make_model(graph_def, opset_imports=[onnx.helper.make_opsetid("", 22)]) # Explicitly set opset to 13
    model.ir_version = 10 
    onnx.checker.check_model(model)

    # Save the model
    onnx.save(model, "maxpool.onnx")

    # Load and run the model with ONNX Runtime
    session = ort.InferenceSession("maxpool.onnx")
    return session

def do_maxpool(x, session):
    # Run inference
    output = session.run(None, {input1_name: x})

    x_f = (np.array2string(x, separator=',', max_line_width=np.inf).replace('\n', '\n'))
    y_f = (np.array2string(output[0], separator=',', max_line_width=np.inf).replace('\n', '\n'))
    indices_f = (np.array2string(output[1], separator=',', max_line_width=np.inf).replace('\n', '\n'))

    # Display results
    print(f"X = \n{x_f}")
    print(f"Y = \n{y_f}")
    print(f"Indices = \n{indices_f}")

def compute_MaxPool_output_shape(input_shape, kernel_shape, pads, dilation, strides):

    dx0, dx1, dx2, dx3 = input_shape
    dw0, dw1 = kernel_shape


    myalpha = dx2 + pads[0] + pads[2]
    mybeta = dx3 + pads[1] + pads[3]
    mytheta = (dilation[0] * (dw0 - 1)) + 1
    mygamma = (dilation[1] * (dw1 - 1)) + 1
#   dY_2 = \left\lfloor{(dX_2 + pad\_shape[0] - dilations[0] * (kernel\_shape[0] - 1) - 1) / (strides[0] + 1)}\right\rfloor
    mydy2 = math.floor(((myalpha - (mytheta)) / strides[0]) + 1)
#   dY_3 = \left\lfloor{(dX_3 + pad\_shape[1] - dilations[1] * (kernel\_shape[1] - 1) - 1) / (strides[1] + 1)} \right\rfloor
    mydy3 = math.floor(((mybeta - (mygamma)) / strides[1]) + 1)
    
    output_shape = [dx0, dx1, mydy2, mydy3]

    return output_shape


def pad_4d_spatial_js(tensor, paddings):
    """
    Pads the last two dimensions (H, W) of a 4D tensor.
    paddings = (top, left, bottom, right)
    """
    N, C, H, W = tensor.shape
    top, left, bottom, right = paddings
    
    # 1. Calculate the new spatial dimensions
    new_H = H + top + bottom
    new_W = W + left + right
    
    # 2. Create the new 4D "canvas" of zeros
    # The Batch (N) and Channel (C) dimensions remain the same
#    padded_tensor = np.zeros((N, C, new_H, new_W), dtype=tensor.dtype)
    padded_tensor = np.full((N, C, new_H, new_W), -np.inf)
    
    # 3. Insert the original tensor into the padded one
    # We use ":" for N and C to select all batches and channels
    # We use slicing for the H and W dimensions
    padded_tensor[:, :, top : top + H, left : left + W] = tensor
    
    return padded_tensor

def extract_max_dilated_from_dilated_window(X_p, b, c, m, n, strides, dilations, dW):
    """
    Extracts the maximum value from a dilated window.
    
    Args:
        X_p: The 2D padded input array.
        m, n: Integer constants representing the current output position.
        strides: Tuple (s_h, s_w) for step size.
        dilations: Tuple (d_h, d_w) for spacing between window elements.
        dW: Tuple (dW0, dW1) for the window height and width.
    """
    dW0, dW1 = dW
    s_h, s_w = strides
    d_h, d_w = dilations
   
    # Calculate the base (top-left) coordinates based on the stride
    base_h = m * s_h
    base_w = n * s_w
    
    # Initialize max_val with the first element of the window (h=0, w=0)
    max_val = X_p[b, c, base_h, base_w]
    
    # Iterate through the window defined by dW0 and dW1
    for h in range(dW0):
        for w in range(dW1):
            # Calculate the current dilated coordinates
            curr_h = base_h + h * d_h
            curr_w = base_w + w * d_w
            
            # Extract the value at these coordinates
            val = X_p[b, c, curr_h, curr_w]
            
            # Update the maximum found so far
            if val > max_val:
                max_val = val
                
    return max_val

def MaxPool(X, kernel_shape, strides, dilation, pads):

    Y_dims = compute_MaxPool_output_shape(X.shape, kernel_shape, pads, dilation, strides)

    Y = np.zeros(Y_dims)

    X_p = pad_4d_spatial_js(X, pads)
    dX_p0, dX_p1, dX_p2, dX_p3 = X_p.shape
    dY_p0, dY_p1, dY_p2, dY_p3 = Y.shape

    xp_f = (np.array2string(X_p, separator=',', max_line_width=np.inf).replace('\n', '\n'))
    print(f"MaxPool: X_p = \n{xp_f}")

    for b in range(dX_p0):
       for c in range(dX_p1): 
         for m in range(dY_p2):
           for n in range(dY_p3):
             Y[b, c, m, n] = extract_max_dilated_from_dilated_window(X_p, b, c, m, n, strides, dilation, kernel_shape)

    Y_f = (np.array2string(Y, separator=',', max_line_width=np.inf).replace('\n', '\n'))
    print(f"MaxPool: Y = \n{Y_f}")


np.set_printoptions(precision=None, floatmode='fixed')

## Nominal Cases

In [7]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
x = np.random.normal(3, 2.5, size=(1, 1, 3, 3))
pads = [0,0,0,0]
kernel_shape = [2,2]
session = create_maxpool_model(x.shape, [1, 1, 2, 2], onnx_type, pads, kernel_shape)
do_maxpool(x, session)
MaxPool(x, kernel_shape, [1,1], [1,1], pads)

X = 
[[[[1.70792822,1.59383029,2.22933891],
   [1.39774388,2.03411151,3.15139065],
   [2.81201102,5.85721996,3.55039159]]]]
Y = 
[[[[2.03411151,3.15139065],
   [5.85721996,5.85721996]]]]
Indices = 
[[[[4,5],
   [7,7]]]]
MaxPool: X_p = 
[[[[1.70792822,1.59383029,2.22933891],
   [1.39774388,2.03411151,3.15139065],
   [2.81201102,5.85721996,3.55039159]]]]
MaxPool: Y = 
[[[[2.03411151,3.15139065],
   [5.85721996,5.85721996]]]]


In [50]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
data = [[[[-np.inf,-np.inf,4.56432533],
          [-np.inf,-np.inf,2.55354471],
          [2.83691720,3.46789489,5.23979851]]]]
x = np.array(data, dtype=np.float64)
pads = [0,0,0,0]
kernel_shape = [2,2]
session = create_maxpool_model(x.shape, [1, 1, 2, 2], onnx_type, pads, kernel_shape)
do_maxpool(x, session)
MaxPool(x, kernel_shape, [1,1], [1,1], pads)

X = 
[[[[      -inf,      -inf,4.56432533],
   [      -inf,      -inf,2.55354471],
   [2.83691720,3.46789489,5.23979851]]]]
Y = 
[[[[-1.79769313e+308, 4.56432533e+000],
   [ 3.46789489e+000, 5.23979851e+000]]]]
Indices = 
[[[[-4, 2],
   [ 7, 8]]]]
MaxPool: X_p = 
[[[[      -inf,      -inf,4.56432533],
   [      -inf,      -inf,2.55354471],
   [2.83691720,3.46789489,5.23979851]]]]
MaxPool: Y = 
[[[[      -inf,4.56432533],
   [3.46789489,5.23979851]]]]


In [10]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
x = np.random.normal(3, 2.5, size=(1, 1, 3, 3))
pads = [1,0,1,0]
kernel_shape = [2,2]
session = create_maxpool_model(x.shape, [1, 1, 4, 2], onnx_type, pads, kernel_shape)
do_maxpool(x, session)
MaxPool(x, kernel_shape, [1,1], [1,1], pads)

X = 
[[[[ 2.41529657, 0.12586645, 5.17877496],
   [ 5.82770299, 3.77328965, 3.51988829],
   [ 1.40679595, 3.95043140,-1.37421443]]]]
Y = 
[[[[2.41529657,5.17877496],
   [5.82770299,5.17877496],
   [5.82770299,3.95043140],
   [3.95043140,3.95043140]]]]
Indices = 
[[[[0,2],
   [3,2],
   [3,7],
   [7,7]]]]
MaxPool: X_p = 
[[[[       -inf,       -inf,       -inf],
   [ 2.41529657, 0.12586645, 5.17877496],
   [ 5.82770299, 3.77328965, 3.51988829],
   [ 1.40679595, 3.95043140,-1.37421443],
   [       -inf,       -inf,       -inf]]]]
MaxPool: Y = 
[[[[2.41529657,5.17877496],
   [5.82770299,5.17877496],
   [5.82770299,3.95043140],
   [3.95043140,3.95043140]]]]


In [49]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
data = [[[[-np.inf,9.57875561,4.56432533],
          [2.72844928,3.54234851,2.55354471],
          [2.83691720,3.46789489,5.23979851]]]]
x = np.array(data, dtype=np.float64)
pads = [1,1,1,1]
kernel_shape = [2,2]
session = create_maxpool_model(x.shape, [1, 1, 4, 4], onnx_type, pads, kernel_shape)
do_maxpool(x, session)
MaxPool(x, kernel_shape, [1,1], [1,1], pads)

X = 
[[[[      -inf,9.57875561,4.56432533],
   [2.72844928,3.54234851,2.55354471],
   [2.83691720,3.46789489,5.23979851]]]]
Y = 
[[[[-1.79769313e+308, 9.57875561e+000, 9.57875561e+000, 4.56432533e+000],
   [ 2.72844928e+000, 9.57875561e+000, 9.57875561e+000, 4.56432533e+000],
   [ 2.83691720e+000, 3.54234851e+000, 5.23979851e+000, 5.23979851e+000],
   [ 2.83691720e+000, 3.46789489e+000, 5.23979851e+000, 5.23979851e+000]]]]
Indices = 
[[[[-4, 1, 1, 2],
   [ 3, 1, 1, 2],
   [ 6, 4, 8, 8],
   [ 6, 7, 8, 8]]]]
MaxPool: X_p = 
[[[[      -inf,      -inf,      -inf,      -inf,      -inf],
   [      -inf,      -inf,9.57875561,4.56432533,      -inf],
   [      -inf,2.72844928,3.54234851,2.55354471,      -inf],
   [      -inf,2.83691720,3.46789489,5.23979851,      -inf],
   [      -inf,      -inf,      -inf,      -inf,      -inf]]]]
MaxPool: Y = 
[[[[      -inf,9.57875561,9.57875561,4.56432533],
   [2.72844928,9.57875561,9.57875561,4.56432533],
   [2.83691720,3.54234851,5.23979851,5.23979851],
 

In [14]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.DOUBLE
data = [[[[-np.inf,9.57875561,4.56432533],
          [2.72844928,3.54234851,2.55354471],
          [2.83691720,3.46789489,5.23979851]]]]
x = np.array(data, dtype=np.float64)
pads = [0,1,0,1]
kernel_shape = [2,2]
session = create_maxpool_model(x.shape, [1, 1, 2, 4], onnx_type, pads, kernel_shape)
do_maxpool(x, session)
MaxPool(x, kernel_shape, [1,1], [1,1], pads)

X = 
[[[[      -inf,9.57875561,4.56432533],
   [2.72844928,3.54234851,2.55354471],
   [2.83691720,3.46789489,5.23979851]]]]
Y = 
[[[[2.72844928,9.57875561,9.57875561,4.56432533],
   [2.83691720,3.54234851,5.23979851,5.23979851]]]]
Indices = 
[[[[3,1,1,2],
   [6,4,8,8]]]]
MaxPool: X_p = 
[[[[      -inf,      -inf,9.57875561,4.56432533,      -inf],
   [      -inf,2.72844928,3.54234851,2.55354471,      -inf],
   [      -inf,2.83691720,3.46789489,5.23979851,      -inf]]]]
MaxPool: Y = 
[[[[2.72844928,9.57875561,9.57875561,4.56432533],
   [2.83691720,3.54234851,5.23979851,5.23979851]]]]


In [46]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.INT8
x = np.random.normal(3, 2.5, size=(1, 1, 3, 3))
x_int8 = np.clip(np.round(x), -128, 127).astype(np.int8)
pads = [0,0,0,0]
kernel_shape = [2,2]
session = create_maxpool_model(x_int8.shape, [1, 1, 2, 2], onnx_type, pads, kernel_shape)
do_maxpool(x_int8, session)
MaxPool(x_int8, kernel_shape, [1,1], [1,1], pads)

X = 
[[[[-1, 3, 4],
   [ 5,-3,10],
   [ 3, 1, 2]]]]
Y = 
[[[[ 5,10],
   [ 5,10]]]]
Indices = 
[[[[3,5],
   [3,5]]]]
MaxPool: X_p = 
[[[[-1.00000000, 3.00000000, 4.00000000],
   [ 5.00000000,-3.00000000,10.00000000],
   [ 3.00000000, 1.00000000, 2.00000000]]]]
MaxPool: Y = 
[[[[ 5.00000000,10.00000000],
   [ 5.00000000,10.00000000]]]]


In [53]:
onnx_type = onnx.TensorProto.INT8
data = [[[[ -12, -13, 5],
          [ -14, -15, 6],
          [ 7, 8, -1]]]]
x = np.array(data, dtype=np.int8)
pads = [0,0,0,0]
kernel_shape = [2,2]
session = create_maxpool_model(x_int8.shape, [1, 1, 2, 2], onnx_type, pads, kernel_shape)
do_maxpool(x, session)
#MaxPool(x_int8, [3,3], [1,1], [1,1], pads)

X = 
[[[[-12,-13,  5],
   [-14,-15,  6],
   [  7,  8, -1]]]]
Y = 
[[[[-12,  6],
   [  8,  8]]]]
Indices = 
[[[[0,5],
   [7,7]]]]


In [54]:
onnx_type = onnx.TensorProto.INT8
data = [[[[ -128, -128, 5],
          [ -128, -128, 6],
          [ 7, 8, -1]]]]
x = np.array(data, dtype=np.int8)
pads = [0,0,0,0]
kernel_shape = [2,2]
session = create_maxpool_model(x_int8.shape, [1, 1, 2, 2], onnx_type, pads, kernel_shape)
do_maxpool(x, session)
#MaxPool(x_int8, [3,3], [1,1], [1,1], pads)

X = 
[[[[-128,-128,   5],
   [-128,-128,   6],
   [   7,   8,  -1]]]]
Y = 
[[[[-128,   6],
   [   8,   8]]]]
Indices = 
[[[[-4, 5],
   [ 7, 7]]]]


In [20]:
# Case N1: BLA BLA BLA
onnx_type = onnx.TensorProto.INT8
x = np.random.normal(3, 2.5, size=(1, 1, 3, 3))
x_int8 = np.clip(np.round(x), -128, 127).astype(np.int8)
pads = [0,1,1,1]
kernel_shape = [2,2]
session = create_maxpool_model(x_int8.shape, [1, 1, 3, 4], onnx_type, pads, kernel_shape)
do_maxpool(x_int8, session)
#MaxPool(x_int8, kernel_shape, [1,1], [1,1], pads)

X = 
[[[[ 1, 0, 3],
   [ 4, 2,-3],
   [ 4, 3, 2]]]]
Y = 
[[[[4,4,3,3],
   [4,4,3,2],
   [4,4,3,2]]]]
Indices = 
[[[[3,3,2,2],
   [3,3,7,8],
   [6,6,7,8]]]]


In [28]:
data = [[[[ -128, -127, 5],
          [ -128, -127, 6],
          [ 7, 8, -128]]]]
x = np.array(data, dtype=np.int8)
pads = [1,1,1,1]
kernel_shape = [2,2]
session = create_maxpool_model(x_int8.shape, [1, 1, 4, 4], onnx_type, pads, kernel_shape)
do_maxpool(x, session)
#MaxPool(x_int8, [3,3], [1,1], [1,1], pads)

X = 
[[[[-128,-127,   5],
   [-128,-127,   6],
   [   7,   8,-128]]]]
Y = 
[[[[-128,-127,   5,   5],
   [-128,-127,   6,   6],
   [   7,   8,   8,   6],
   [   7,   8,   8,-128]]]]
Indices = 
[[[[-4, 1, 2, 2],
   [-4, 1, 5, 5],
   [ 6, 7, 7, 5],
   [ 6, 7, 7,-4]]]]


In [31]:
onnx_type = onnx.TensorProto.UINT8
x = np.random.normal(3, 2.5, size=(1, 1, 3, 3))
x_uint8 = np.clip(np.round(x), 0, 127).astype(np.uint8)
pads = [0,0,0,0]
kernel_shape = [2,2]
session = create_maxpool_model(x_uint8.shape, [1, 1, 2, 2], onnx_type, pads, kernel_shape)
do_maxpool(x_uint8, session)
#MaxPool(x_int8, kernel_shape, [1,1], [1,1], pads)

X = 
[[[[8,5,2],
   [7,0,7],
   [3,3,3]]]]
Y = 
[[[[8,7],
   [7,7]]]]
Indices = 
[[[[0,5],
   [3,5]]]]


In [56]:
onnx_type = onnx.TensorProto.UINT8
data = [[[[ 0, 0, 5],
          [ 0, 0, 6],
          [ 7, 8, 0]]]]
x = np.array(data, dtype=np.uint8)
pads = [0,0,0,0]
kernel_shape = [2,2]
session = create_maxpool_model(x_uint8.shape, [1, 1, 2, 2], onnx_type, pads, kernel_shape)
do_maxpool(x, session)
#MaxPool(x_int8, [3,3], [1,1], [1,1], pads)

X = 
[[[[0,0,5],
   [0,0,6],
   [7,8,0]]]]
Y = 
[[[[0,6],
   [8,8]]]]
Indices = 
[[[[-4, 5],
   [ 7, 7]]]]


In [57]:
onnx_type = onnx.TensorProto.UINT8
data = [[[[ 0, 1, 5],
          [ 1, 1, 6],
          [ 7, 8, 0]]]]
x = np.array(data, dtype=np.uint8)
pads = [1,1,1,1]
kernel_shape = [2,2]
session = create_maxpool_model(x_uint8.shape, [1, 1, 4, 4], onnx_type, pads, kernel_shape)
do_maxpool(x, session)
#MaxPool(x_int8, [3,3], [1,1], [1,1], pads)

X = 
[[[[0,1,5],
   [1,1,6],
   [7,8,0]]]]
Y = 
[[[[0,1,5,5],
   [1,1,6,6],
   [7,8,8,6],
   [7,8,8,0]]]]
Indices = 
[[[[-4, 1, 2, 2],
   [ 3, 1, 5, 5],
   [ 6, 7, 7, 5],
   [ 6, 7, 7,-4]]]]


In [36]:
data = [[[[ 0, 1, 5],
          [ 1, 1, 6],
          [ 7, 8, 0]]]]
x = np.array(data, dtype=np.uint8)
pads = [0,0,0,0]
kernel_shape = [2,2]
session = create_maxpool_model(x_uint8.shape, [1, 1, 2, 2], onnx_type, pads, kernel_shape)
do_maxpool(x, session)
#MaxPool(x_int8, [3,3], [1,1], [1,1], pads)

X = 
[[[[0,1,5],
   [1,1,6],
   [7,8,0]]]]
Y = 
[[[[1,6],
   [8,8]]]]
Indices = 
[[[[1,5],
   [7,7]]]]
